In [1]:
import requests
import pathlib
import pandas as pd

In [2]:

productGuids = [
    "07d1c32e-8f8d-41ba-b0c1-6755b12261fb","08696a82-139e-43c8-b54c-dedc568f0073","086eb3e4-89d0-4f46-b6ed-11a8834deb3e","086d9f2e-cc52-4086-a898-e540b862e5c0","085d3421-a6b2-4448-8242-7a59f7e5226a","0859d695-fa50-4a51-8c30-2a73f9453991","085a3eb2-9c57-4825-a171-bef771aa30f4","083f1e90-47fb-414e-a228-f89ab678b646","08470d8e-e7bd-4062-8d2c-5c7057c83289","083d7b1a-1ebb-4cd5-9576-1acabcfbba44","081c8426-ae3f-4f36-a0af-ef7730708352","081b39c0-1ac1-405c-9f66-747e2c74e305","07f25bf3-9f78-48dd-9cbe-46349d86987b","07d0a0c5-985b-482f-ba1d-c8cdfcd0032d","07d0fb5f-3240-4bfd-8d55-eb9bb363a55c"
]
# workflowId = input("Enter Workflow ID: ")
# auth = input("Enter Auth Code: ")
workflowId = "c0c48edf-2645-4271-b8b6-cfa33d720876"
auth = "EN AkVOZZ4SaRThrEuRm/U8cL6KKAhwbWFyYmliaXT9eVFP6tkBdT1ees/v2QECAAAAEkVOLlJvb3RQZXJtaXNzaW9ucwMxMjYAAA9FTi5Vc2VyRnVsbE5hbWULUGFuIE1hcmJpYmkAAIscXAbqdbcCteYGlP6CJ1TLYqVi3vNx2gUnkVyUYnEM3zMiNKM+CwAmhr2AKW9cUFqw0UcwwBXL2FH87+e76hKPok65AP0Tf9+7P8fPZd/uV1iDH600hSxVf/w8LkFQc1Uh1JXlexAzQEDRXhvLYU9C5UR73ITqWiX+L6mJYoxl" 
headers = {
    "User-Agent": "PostmanRuntime/y.32.3",
    "Accept": "*/*",
    "Accept-Encoding": "gzip,deflate,br" ,
    "Connection" : "keep-alive",
    "Authorization": auth
    }
env = "api.syndigo.com"

In [65]:
import json

dataowner = "c7407733-1be6-452b-82b1-95a89c0cb2a1"

wf_name_and_statuses = f"https://{env}/api/workflow/workflowgrain/bydataowner/{dataowner}"


name_response = requests.get(wf_name_and_statuses, headers=headers)

wf_statuses = []
step_ids = []
step_desc = []

if name_response.status_code == 200:
    response_json = name_response.json()
    step_names = response_json[workflowId]
    for step_id, step_data in step_names.get("StepNames", {}).items():
        step_ids.append(step_id)
        step_desc.append(step_data.get("InvariantCultureDescription"))
    steps_with_names = {step_id : desc for step_id, desc in zip(step_ids, step_desc)}
else:
    wf_statuses.append("No Statuses")


def statusId_to_statusName(step_id):
    if step_id in steps_with_names:
        return steps_with_names[step_id]
    else:
        return step_id



In [69]:
statusId_to_statusName("2621282c-fce5-4abf-bece-b301278327c8")
# steps_with_names

'Brand'

In [70]:
timeStamp = []
statuses = []
wf_version = []
instaceIds = []

from datetime import datetime
import pytz

cst_timezone = pytz.timezone('America/Chicago')


for product in productGuids:
    status_search_url = f"https://{env}/api/workflow/WorkflowGrain/status/{workflowId}/{product}"
    response = requests.get(status_search_url, headers=headers)
    if response.status_code == 200:
        json_response = response.json()
        timestamp_response = json_response["Timestamp"]
        timestamp_without_fractional_seconds = timestamp_response.split('.')[0]
        parsed_datetime = datetime.fromisoformat(timestamp_without_fractional_seconds)
        cst_datetime = parsed_datetime.astimezone(cst_timezone)
        readable_format = cst_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")
        timeStamp.append(readable_format)
        status_response = json_response["Statuses"]
        stat_names = []
        for status in status_response:
            name = statusId_to_statusName(status)
            stat_names.append(name)
        statuses.append(stat_names)
        wf_version.append(json_response["WorkflowVersion"])
        instaceIds.append(json_response["WorkflowInstanceId"])
    else:
        timeStamp.append("Null")
        statuses.append("Null")
        wf_version.append("Null")
        instaceIds.append("Null")

data = {
    "Product ID" : productGuids,
    "Updated" : timeStamp,
    "Status" : statuses,
    "Instance ID": instaceIds,
    "Version" : wf_version
}

workflow_statuses = pd.DataFrame(data)



In [71]:
workflow_statuses

,Product ID,Updated,Status,Instance ID,Version
0,07d1c32e-8f8d-41ba-b0c1-6755b12261fb,2023-09-13 05:14:10 CDT,"[Unique Sent, Physical]",ed8c4bf9-f507-4dff-aa05-e60156d77ede,41
1,08696a82-139e-43c8-b54c-dedc568f0073,2023-09-09 05:09:22 CDT,"[Unique Sent, Digital, WaitingReject]",f69526ca-f436-4625-9547-320e0b62ce3d,41
2,086eb3e4-89d0-4f46-b6ed-11a8834deb3e,2023-09-12 16:20:32 CDT,[Excluded Marketplace Items],0e9459f1-ebe3-475c-94df-17eec44f27b0,41
3,086d9f2e-cc52-4086-a898-e540b862e5c0,2023-09-13 05:14:13 CDT,"[Unique Sent, Physical]",18dee6f2-041e-4a87-90cb-443e2934c091,41
4,085d3421-a6b2-4448-8242-7a59f7e5226a,2023-09-09 05:09:22 CDT,"[Unique Sent, Digital, WaitingReject]",6cad499a-1444-4a1b-bfec-f79f9e19fc4c,41
5,0859d695-fa50-4a51-8c30-2a73f9453991,2023-09-13 05:14:10 CDT,"[Unique Sent, Physical]",7be92c89-b032-4db6-ba8d-a8cb9aa73ff6,41
6,085a3eb2-9c57-4825-a171-bef771aa30f4,2023-09-09 05:09:39 CDT,"[Unique Sent, Brand]",642d5828-991d-4696-8628-1b78ce4c5cd5,41
7,083f1e90-47fb-414e-a228-f89ab678b646,2023-09-02 05:09:15 CDT,"[Unique Sent, Brand]",7b9affb9-8b16-48f0-8286-d76efce6658e,40
8,08470d8e-e7bd-4062-8d2c-5c7057c83289,2023-09-13 05:14:22 CDT,"[Unique Sent, Digital, WaitingReject]",af5dd21b-1263-4f47-9524-995694211462,41
9,083d7b1a-1ebb-4cd5-9576-1acabcfbba44,2023-09-13 05:14:21 CDT,"[Unique Sent, Digital, WaitingReject]",8be6d565-7d57-41e9-bfd5-cccf529a2dd0,41
